# Packages

In [1]:
import pandas as pd 
import numpy as np
import re 

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display


import dash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'



# Data Ingestion

In [2]:
df = pd.read_csv('Data/State_of_Iowa_Salary_Book.csv')
df.head()

,Fiscal Year,Department,Agency/Institution,Name,Gender,Place of Residence,Position,Base Salary,Base Salary Date,Total Salary Paid,Travel & Subsistence
0,2019,"Administrative Services, Department of",NaN,"ADAMS, BRANDON D",M,MARION,ELECTRICIAN,"2,594.40 BW",Jul 2019,68724.22,0.00
1,2019,"Administrative Services, Department of",NaN,"ADAMS, CAROL L",F,MITCHELL,HR PROFESSIONAL 2,"3,513.60 BW",Jul 2019,89447.20,239.00
2,2019,"Administrative Services, Department of",NaN,"ADAMS, JEANNIE R",F,POLK,PUBLIC SERVICE SUPERVISOR,"3,045.60 BW",Jul 2019,23475.20,0.00
3,2019,"Administrative Services, Department of",NaN,"AINGER, STEVEN J",M,DALLAS,HR PROGRAM COORD,"3,377.60 BW",Jul 2019,84764.00,1205.93
4,2019,"Administrative Services, Department of",NaN,"ANSTEY, LYNN M",F,POLK,ACCOUNTING TECHNICIAN 2,"2,048.80 BW",Jul 2019,52146.40,0.00


In [3]:
# Removal of unnecessary columns
df.drop(columns = ['Agency/Institution','Base Salary', "Name"], inplace = True)

In [4]:
# Handling of null values
df.dropna(subset=['Total Salary Paid', 'Gender'], inplace = True)

df['Place of Residence'] = df['Place of Residence'].fillna('Unknown')
df['Place of Residence'] = df['Place of Residence'].replace(['**', '-1', '00', '74820150619'], 'Unknown')

df['Gender'] = df['Gender'].replace(['m', 'H', 'FM'], ['M','M','F'])
df['Gender'] = df['Gender'].replace(['*', 'U'], np.nan)
df = df.dropna(subset=['Gender'])

df['Total Salary Paid'] = df['Total Salary Paid'].apply(lambda x: max(float(x), 0)).replace(0,np.nan)
df = df.drop(df[(df['Total Salary Paid'] >= 0) & (df['Total Salary Paid'] < 1000)].index)


#### Column: Place of Residence

In [6]:
# Cleaning the columns values 

df['Place of Residence'] = df['Place of Residence'].apply(lambda x: ' '.join(word.capitalize() for word in x.split()))

#Function to remove the numbers and keep only the names of the cities.
def remove_numbers(valor):
    return re.sub(r'\d+', '', valor).strip('-')

# Applying the function
df['Place of Residence'] = df['Place of Residence'].apply(remove_numbers)


# Auxiliary dictionaries 

from city_to_state import city_to_state_dict

cities_set = set(city_to_state_dict.keys())

auxiliary_data = pd.read_csv('data/georef-united-states-of-america-county.csv', sep = ';')

auxiliary_data.drop(columns = ['Geo Point', 'Geo Shape', 'Year', 'Official Code State','Iso 3166-3 Area Code', 'Type',
       'Name with legal/statistical area description', 'County FIPS Code',
       'County GNIS Code', 'Official Code County'], inplace = True)

county_state_dict = dict(zip(auxiliary_data['Official Name County'], auxiliary_data['Official Name State']))
county_state_dict

#New Column: State

def find_state(place_of_residence, city_state_dict, county_state_dict):
    
    if place_of_residence in city_state_dict:
        return city_state_dict[place_of_residence]
    elif place_of_residence in county_state_dict:
        return county_state_dict[place_of_residence]
    else:
        return 'Unknown'

df['State'] = df['Place of Residence'].apply(lambda x: find_state(x, city_to_state_dict, county_state_dict))

state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}


df['State'] = df['State'].map(state_abbrev)
df = df.dropna(subset=['State'])

#### Column Department

In [7]:
df['Department'] = df['Department'].str.replace(', Department on', ', Department of').str.replace(', Department for', ', Department of').str.strip()

def reverse_and_remove_comma(department_name):
    parts = department_name.split(', ')
    reversed_name = ', '.join(reversed(parts))
    return reversed_name.replace(',', '')

df['Department'] = df['Department'].apply(reverse_and_remove_comma)

df.head()

,Fiscal Year,Department,Gender,Place of Residence,Position,Base Salary Date,Total Salary Paid,Travel & Subsistence,State
0,2019,Department of Administrative Services,M,Marion,ELECTRICIAN,Jul 2019,68724.22,0.00,OH
1,2019,Department of Administrative Services,F,Mitchell,HR PROFESSIONAL 2,Jul 2019,89447.20,239.00,SD
2,2019,Department of Administrative Services,F,Polk,PUBLIC SERVICE SUPERVISOR,Jul 2019,23475.20,0.00,WI
3,2019,Department of Administrative Services,M,Dallas,HR PROGRAM COORD,Jul 2019,84764.00,1205.93,TX
4,2019,Department of Administrative Services,F,Polk,ACCOUNTING TECHNICIAN 2,Jul 2019,52146.40,0.00,WI


# Some analysis

### Department comparison over the years


In [21]:
# Aggregating total and mean
df_departments_year= df.groupby(['Department', 'Fiscal Year'])['Total Salary Paid'].agg(['sum', 'mean']).reset_index()
df_departments_year.columns = ['Department', 'Fiscal Year', 'Total Salary Paid', "Mean of total salary paid"]
df_departments_year['Total Salary Paid'] = round(df_departments_year['Total Salary Paid'], 2)


df_departments_year.head(10)

,Department,Fiscal Year,Total Salary Paid,Mean of total salary paid
0,Board of Educational Examiners,2019,1064764.47,76054.605000
1,Board of Educational Examiners,2020,1046149.52,69743.301333
2,Board of Educational Examiners,2021,1043112.00,74508.000000
3,Board of Educational Examiners,2022,1095493.77,68468.360625
4,Board of Educational Examiners,2023,1210393.34,75649.583750
5,Board of Parole,2007,793648.36,34506.450435
6,Board of Parole,2008,768126.19,38406.309500
7,Board of Parole,2009,749026.81,39422.463684
8,Board of Parole,2010,735342.94,36767.147000
9,Board of Parole,2011,621301.10,31065.055000


In [9]:
# Graph 
def update_plot(selected_departments):
    if not selected_departments:
        return  

    df_selected = df_departments_year[df_departments_year['Department'].isin(selected_departments)]
    
    # Boxplot creation
    fig = px.box(df_selected, 
                 x='Department', 
                 y='Total Salary Paid', 
                 color='Department', 
                 title=f'Total Salary Paid for {", ".join(selected_departments)}',
                 labels={'Total Salary Paid': 'Total Salary Paid', 'Department': 'Department'},
                 height=600,
                 width=1000)

    fig.update_layout(xaxis_title='Department', 
                      yaxis_title='Total Salary Paid', 
                      xaxis_tickangle=-45,
                      legend_title='Department')

    fig.show()

# List of unique departments
unique_departments = df_departments_year['Department'].unique()

# Creation of the dropdown widget to select departments
department_dropdown = widgets.SelectMultiple(
    options=unique_departments,
    description='Department:',
    disabled=False,
    layout=widgets.Layout(width='50%'),
    value=[unique_departments[0]]
)

# Interactive interface
interactive_plot = widgets.interactive(update_plot, selected_departments=department_dropdown)

# Displays the widget and interactive graphic
display(interactive_plot)

interactive(children=(SelectMultiple(description='Department:', index=(0,), layout=Layout(width='50%'), option…

In [27]:
def update_plot_2(selected_departments):
    if not selected_departments:
        return  

    df_selected = df_departments_year[df_departments_year['Department'].isin(selected_departments)]
    
    # Creation of a line graph
    fig = px.line(df_selected, 
                 x='Fiscal Year', 
                 y='Total Salary Paid', 
                 color='Department', 
                 markers=True,
                 title=f'Total Salary Paid for {", ".join(selected_departments)}',
                 labels={'Total Salary Paid': 'Total Salary Paid', 'Fiscal Year': 'Year'},
                 height=600,
                 width=900)

    fig.update_layout(xaxis_title='Fiscal year', 
                      yaxis_title='Total Salary Paid', 
                      xaxis_tickangle=-45,
                      legend_title='Department')
    department_means = df_selected.groupby('Department')['Total Salary Paid'].mean().reset_index()    
    for _, row in department_means.iterrows():
        fig.add_shape(
        type='line',
        x0=df_selected['Fiscal Year'].min(),
        x1=df_selected['Fiscal Year'].max(),
        y0=row['Total Salary Paid'],
        y1=row['Total Salary Paid'],
        line=dict(color='Red', dash='dash'),
        xref='x',
        yref='y'
    )
    # Adicionar anotação para cada linha de média
    fig.add_annotation(
        x=df_selected['Fiscal Year'].min(),
        y=row['Total Salary Paid'],
        text=f'{row["Department"]} Mean: {row["Total Salary Paid"]:.2f}',
        showarrow=False,
        yshift=10,
        font=dict(color='Red')
    )


    fig.show()

# Criação do widget de dropdown para selecionar departamentos
department_SelectMultiple_2 = widgets.SelectMultiple(
    options=unique_departments,
    description='Department:',
    disabled=False,
    layout=widgets.Layout(width='50%'),
    value=[unique_departments[0]]
)

# Use widgets.interactive para criar a interface interativa
interactive_plot = widgets.interactive(update_plot_2, selected_departments=department_SelectMultiple_2)

# Exibe o widget e o gráfico interativo
display(interactive_plot)

interactive(children=(SelectMultiple(description='Department:', index=(0,), layout=Layout(width='50%'), option…

### Department comparison over the state of residence

In [11]:
# Aggregating mean and median

df_state_mean = df.groupby('State')['Total Salary Paid'].mean().reset_index()
df_state_median = df.groupby('State')['Total Salary Paid'].median().reset_index()
df_state = pd.merge(df_state_mean, df_state_median, on='State', suffixes=('_mean', '_median'))
df_state.columns = ['State', 'Mean Total Salary Paid', 'Median Total Salary Paid']
df_state[['Mean Total Salary Paid', 'Median Total Salary Paid']] = round(df_state[['Mean Total Salary Paid', 'Median Total Salary Paid']], 2)

# Coefficient of variation
df_state_cv = round(df.groupby('State')['Total Salary Paid'].std() / df.groupby('State')['Total Salary Paid'].mean(),2)
df_state_cv = df_state_cv.reset_index(name='CV')

# New DataFrame 
df_state = df_state.merge(df_state_cv, on= 'State')

# Cleaning the data


df_state.head()

,State,Mean Total Salary Paid,Median Total Salary Paid,CV
0,AK,21364.27,14690.10,1.03
1,AL,41919.89,44142.36,0.75
2,AR,44001.40,46465.65,0.62
3,AZ,33763.17,23430.00,1.22
4,CA,39180.37,42381.39,0.80


In [12]:
def update_plot_2(metric, selected_state):
    df_selected = df_departments_year[df_departments_year['Department'] == selected_state]
    
    if metric == 'Mean':
        y_column = 'Mean Total Salary Paid'
        y_label = 'Mean of Total Salary Paid'
    elif metric == 'Median':
        y_column = 'Median Total Salary Paid'
        y_label = 'Median of Total Salary Paid'
     
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df_state[df_state['State'].isin(selected_state)], 
                 x='Fiscal Year', y=y_column, hue='Department', palette='tab10')
    plt.title(f'{metric} of Total Salary paid for choosen Departments')
    plt.xlabel('Fiscal year')
    plt.ylabel(y_label)
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

# List of unique departments
unique_departments = df_departments_year['Department'].unique()

# Create dropdown widget to choose department
department_dropdown = widgets.SelectMultiple(
    options=unique_departments,
    description='Department:',
    disabled=False,
)


# Create dropdown widget to choose the metric
metric_toogle = widgets.ToggleButtons(
    options=['Mean', 'Median', 'CV'],
    description='Measure:',
    disabled=False,
)

widgets.interactive(update_plot, metric=metric_toogle, selected_departments=department_dropdown)

interactive(children=(SelectMultiple(description='Department:', options=('Board of Educational Examiners', 'Bo…

# Analysis by Gender

In [13]:
df2= df.groupby(['Department', 'State', 'Gender'])['Total Salary Paid'].sum().reset_index()
df2.columns = ['Department', 'State','Gender', 'Total Salary Paid']
df2.head(10)

,Department,State,Gender,Total Salary Paid
0,Board of Educational Examiners,IA,F,98952.80
1,Board of Educational Examiners,IL,F,218121.40
2,Board of Educational Examiners,IN,F,228260.44
3,Board of Educational Examiners,OH,M,927430.50
4,Board of Educational Examiners,TX,F,119200.33
5,Board of Educational Examiners,TX,M,463715.20
6,Board of Educational Examiners,WI,F,2288894.76
7,Board of Educational Examiners,WI,M,1081687.75
8,Board of Educational Examiners,WV,F,33649.92
9,Board of Parole,CA,F,8785.87


In [14]:
gender_salary = df2.groupby('Gender')['Total Salary Paid'].sum().reset_index()

fig = px.pie(gender_salary, values='Total Salary Paid', names='Gender', 
             title='Total Salary Paid by Gender', 
             labels={'Gender': 'Gender', 'Total Salary Paid': 'Total Salary Paid'},
        color_discrete_sequence = ['red','blue'])

fig.show()

In [15]:
def update_plot_3(selected_departments):
    if not selected_departments:
        return 

    df2_selected = df2[df2['Department'].isin(selected_departments)]

    # First Graph: Total Salary Paid by gender
    fig1 = px.histogram(
        df2_selected,
        x='Department',
        y='Total Salary Paid',
        color='Gender',
        histfunc = 'avg',
        barmode='group',
        title=f'Comparison of Total Salary Paid for {", ".join(selected_departments)} by Gender',
        labels={'Total Salary Paid': 'Average of total Salary Paid', 'Gender': 'Gender'}
    )

    # Second graph: Piechart with total people
    gender_counts = df2_selected['Gender'].value_counts().reset_index()
    gender_counts.columns = ['Gender', 'Count']

    fig2 = px.pie(
        gender_counts,
        names='Gender',
        values='Count',
        title='Total Number of People by Gender'
    )

    # Combining the graphs
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Average of total Salary Paid by Gender', 'Total of employees'), specs=[[{"type": "xy"}, {"type": "domain"}]])

    for trace in fig1['data']:
        fig.add_trace(trace, row=1, col=1)

    for trace in fig2['data']:
        fig.add_trace(trace, row=1, col=2)

    fig.update_layout(height=600, width=800)
    fig.show()

    
df2_unique_departments = df2['Department'].unique()

department_SelectMultiple_3 = widgets.SelectMultiple(
    options=df2_unique_departments,
    description='Department:',
    disabled=False,
    layout=widgets.Layout(width='50%'),
    value=[df2_unique_departments[0]]
)

interactive_plot = widgets.interactive(update_plot_3, selected_departments=department_SelectMultiple_3)

display(interactive_plot)

interactive(children=(SelectMultiple(description='Department:', index=(0,), layout=Layout(width='50%'), option…

In [16]:
def update_quartile_plot(selected_departments):
    if not selected_departments:
        return  # Evita erro se nenhuma seleção for feita

    df2_selected = df2[df2['Department'].isin(selected_departments)]

    # Calcular os quartis para 'Total Salary Paid'
    Q1 = df2_selected['Total Salary Paid'].quantile(0.25)
    Q3 = df2_selected['Total Salary Paid'].quantile(0.75)

    # Filtrar os dados para abaixo do 1º quartil e acima do 3º quartil
    below_Q1 = df2_selected[df2_selected['Total Salary Paid'] < Q1]
    above_Q3 = df2_selected[df2_selected['Total Salary Paid'] > Q3]

    # Criar barplots para cada grupo
    below_Q1_counts = below_Q1.groupby(['Department', 'Gender']).size().reset_index(name='Count')
    above_Q3_counts = above_Q3.groupby(['Department', 'Gender']).size().reset_index(name='Count')

    # Combinar os dados em um único DataFrame com uma nova coluna para o quartil
    below_Q1_counts['Quartile'] = 'Below Q1'
    above_Q3_counts['Quartile'] = 'Above Q3'
    combined_counts = pd.concat([below_Q1_counts, above_Q3_counts])

    # Criar o gráfico de barras
    fig = px.bar(
        combined_counts,
        x='Department',
        y='Count',
        color='Gender',
        barmode='group',
        facet_col='Quartile',
        title='Count of People Below Q1 and Above Q3 by Gender and Department',
        labels={'Count': 'Count', 'Gender': 'Gender'}
    )

    fig.update_layout(height=600, width=1000)
    fig.show()

# Criação do widget de dropdown para selecionar departamentos
department_SelectMultiple_Quartile = widgets.SelectMultiple(
    options=df2_unique_departments,
    description='Department:',
    disabled=False,
    layout=widgets.Layout(width='50%'),
    value=[df2_unique_departments[0]]
)

# Use widgets.interactive para criar a interface interativa
interactive_quartile_plot = widgets.interactive(update_quartile_plot, selected_departments=department_SelectMultiple_Quartile)

# Exibe os widgets e o gráfico interativo
display(interactive_quartile_plot)

interactive(children=(SelectMultiple(description='Department:', index=(0,), layout=Layout(width='50%'), option…

In [17]:
df_state_gender = df.groupby(['State', 'Gender'])['Total Salary Paid'].sum().reset_index()
df_state_department = df.groupby(['State', 'Department'])['Total Salary Paid'].sum().reset_index()


app = dash.Dash(__name__)

# Layout do app
app.layout = html.Div([
    dcc.Dropdown(
        id='department-dropdown',
        options=[
            {'label': department, 'value': department} for department in df['Department'].unique()
        ],
        value=df['Department'].unique()[0],  # Valor inicial do dropdown será o primeiro valor único de 'Department'
        style={'width': '50%'}  # Define a largura do dropdown
    ),
    dcc.Dropdown(
        id='gender-select',
        options=[
            {'label': gender, 'value': gender} for gender in df['Gender'].unique()
        ],
        value=df['Gender'].unique()[0],  # Valor inicial do dropdown será o primeiro valor único de 'Gender'
        style={'width': '50%'} # Define a largura do dropdown
    ),
    dcc.Graph(id='choropleth-map', style={'height': '80vh'})  # Ajuste a altura do mapa aqui
])

# Callback para atualizar o mapa com base na escolha do dropdown
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('department-dropdown', 'value'),
     Input('gender-select', 'value')]
)
def update_map(department_value, gender_value):
    df_plot = df[df['Department'] == department_value]
    df_plot = df_plot[df_plot['Gender'] == gender_value]
    
    # Verifica se o DataFrame tem dados suficientes
    if df_plot.empty:
        return px.choropleth(locations=[], locationmode='USA-states')
    
    fig = px.choropleth(df_plot,
                        locations='State',
                        locationmode='USA-states',
                        color='Total Salary Paid',
                        hover_name='State',
                        hover_data={'Total Salary Paid': True},
                        title=f'Total Salary Paid by {department_value} for {gender_value}',
                        scope='usa')

    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},  # Remove as margens para maximizar o tamanho do mapa
        height=600  # Ajuste a altura do mapa aqui
    )
    
    return fig

# Roda o servidor Dash
if __name__ == '__main__':
    app.run_server(debug=True)